In [33]:
import pyodbc 


Connection Objects
Connection objects should respond to the following methods.

.close(): Close the connection. 
Aradaki bağlantıyı kapatır

.commit(): Commit any pending transaction to the database.
Bekleyen herhangi bir işlemi veritabanına yaptırır

.rollback(): In case a database does provide transactions this method causes the database to roll back to the start of any pending transaction.

Bir veritabanının işlem sağlaması durumunda, bu yöntem veritabanının bekleyen herhangi bir işlemin başlangıcına geri dönmesine neden olur.

.cursor(): Return a new Cursor Object using the connection.

Cursor Attributes

.description: This read-only attribute is a sequence of 7-item sequences: name, type_code, display_size, internal_size, precision, scale, null_ok
Each of these sequences contains information describing one result column.

.rowcount: This read-only attribute specifies the number of rows that the last 

.execute() produced. The attribute is -1 in case no .execute() has been performed on the cursor.

Cursor methods

.callproc( procname [, parameters ] ): Call a stored database procedure with the given name.

.close(): Close the cursor.

The cursor will be unusable from this point forward; an Error (or subclass) exception will be raised if any operation is attempted with the cursor.

.execute(operation [, parameters]): Prepare and execute a database operation (query or command).

.executemany( operation, seq_of_parameters ): Prepare a database operation (query or command) and then execute it against all parameter sequences or mappings found in the sequence seq_of_parameters.


.fetchone(): Fetch the next row of a query result set, returning a single sequence, or None when no more data is available.

An Error (or subclass) exception is raised if the previous call to .execute() did not produce any result set or no call was issued yet.

.fetchmany([size=cursor.arraysize]): Fetch the next set of rows of a query result, returning a sequence of sequences (e.g. a list of tuples). An empty sequence is returned when no more rows are available.

An Error (or subclass) exception is raised if the previous call to .execute() did not produce any result set or no call was issued yet.

.fetchall(): Fetch all (remaining) rows of a query result, returning them as a sequence of sequences (e.g. a list of tuples). Note that the cursor's array size attribute can affect the performance of this operation.

An Error (or subclass) exception is raised if the previous call to .execute() did not produce any result set or no call was issued yet.

In [37]:
#database bağlanma kodu
#bu şekilde bağlanma gerçekleşiyor
def create_server_connection(driver, server, database, user, password):
    conn = None
    try:
        conn = pyodbc.connect(DRIVER = driver, SERVER = server, DATABASE = database, UID = user, PWD = password)
        print("MS SQL Server Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return conn


driver = '{ODBC Driver 17 for SQL Server}'
server = 'localhost' 
database = 'master' 
user = 'sa' 
password = 'Deneme01'
conn = create_server_connection(driver, server, database, user, password)

MS SQL Server Database connection successful


In [39]:
#yeni database oluşturma
def create_database(conn, create_database_query):
    csr = conn.cursor()
    try:
        csr.execute(create_database_query)
        print("Database created successfully")
    except Error as err:
        print(f"Error: '{err}'")
    

conn.autocommit = True
create_database_query = 'CREATE DATABASE TestDB'
create_database(conn, create_database_query)

NameError: name 'Error' is not defined

In [40]:
#yeni bir query'yi execute etme
def execute_query(conn, query):
    csr = conn.cursor()
    try:
        csr.execute(query)
        conn.commit()
        print("Query successful")
    
    except Error as err:
        print(f"Error: '{err}'")


query ="USE SampleRetail"
execute_query(conn, query)

# Query successful

query = """SELECT product_name, list_price
	   FROM product.product"""
execute_query(conn, query)

Query successful
Query successful


In [41]:
csr = conn.cursor()
csr.execute("SELECT TOP 5 product_name, list_price FROM product.product")
row = csr.fetchone()
if row:
    print(row)
#Burası sadece tek row getiriyor

('DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 1710 1715 1730 1735 1750 1755 1955 3000 3005 A100 M30X M35X - Black', Decimal('23.99'))


In [42]:
csr.execute("SELECT TOP 5 product_name, list_price FROM product.product")
row = csr.fetchone()
print('list_price:', row[1]) # access by column index (zero-based)
print('list_price:', row.list_price) # access by name

list_price: 23.99
list_price: 23.99


In [43]:
csr.execute("SELECT TOP 5 product_name, list_price FROM product.product")
while True:
    row = csr.fetchone()
    if not row:
        break
    print('product_name:', row.product_name)

product_name: DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 1710 1715 1730 1735 1750 1755 1955 3000 3005 A100 M30X M35X - Black
product_name: NS-SP1800BL 5.1-Channel Home Theater System (Black)
product_name: Acoustimass 6 Series V Home Theater Speaker System (Black)
product_name: Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3000MHz C15 Desktop Memory Kit - Black (CMK16GX4M2B3000C15)
product_name: Details About Samsung Gear Iconx 2018 Edition Cordfree Fitness Earbuds Black (us Version)


All SQL statements are executed using the Cursor execute() function. If the statement returns rows, such as a select statement, you can retrieve them using the Cursor fetch functions - fetchone(), fetchall(), fetchmany(). If there are no rows, fetchone() will return None, whereas fetchall() and fetchmany() will both return empty lists.

In [44]:
csr.execute("SELECT TOP 5 product_name, list_price FROM product.product")
rows = csr.fetchall()
for row in rows:
    print(row.product_name, row.list_price)

DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 1710 1715 1730 1735 1750 1755 1955 3000 3005 A100 M30X M35X - Black 23.99
NS-SP1800BL 5.1-Channel Home Theater System (Black) 136.99
Acoustimass 6 Series V Home Theater Speaker System (Black) 599.00
Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3000MHz C15 Desktop Memory Kit - Black (CMK16GX4M2B3000C15) 151.99
Details About Samsung Gear Iconx 2018 Edition Cordfree Fitness Earbuds Black (us Version) 199.99


In [45]:
csr.execute("SELECT TOP 5 product_name, list_price FROM product.product")
for row in csr:
    print(row.product_name, row.list_price)

#or just:

for row in csr.execute("SELECT TOP 5 product_name, list_price FROM product.product"):
    print(row.product_name, row.list_price)

DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 1710 1715 1730 1735 1750 1755 1955 3000 3005 A100 M30X M35X - Black 23.99
NS-SP1800BL 5.1-Channel Home Theater System (Black) 136.99
Acoustimass 6 Series V Home Theater Speaker System (Black) 599.00
Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3000MHz C15 Desktop Memory Kit - Black (CMK16GX4M2B3000C15) 151.99
Details About Samsung Gear Iconx 2018 Edition Cordfree Fitness Earbuds Black (us Version) 199.99
DENAQ - AC Adapter for TOSHIBA SATELLITE 1700 1710 1715 1730 1735 1750 1755 1955 3000 3005 A100 M30X M35X - Black 23.99
NS-SP1800BL 5.1-Channel Home Theater System (Black) 136.99
Acoustimass 6 Series V Home Theater Speaker System (Black) 599.00
Corsair Vengeance LPX 16GB (2x8GB) DDR4 DRAM 3000MHz C15 Desktop Memory Kit - Black (CMK16GX4M2B3000C15) 151.99
Details About Samsung Gear Iconx 2018 Edition Cordfree Fitness Earbuds Black (us Version) 199.99


In [46]:
csr.execute("""SELECT TOP 5 first_name, last_name, city FROM sale.customer
               WHERE city = ?""", 'Charlotte')
row = csr.fetchone()
if row:
    print(row)

('William', 'Williams', 'Charlotte')


In [47]:
csr.execute("""SELECT first_name, last_name FROM sale.customer
		where first_name= ? and last_name= ?""", ['Nana', 'Gaines'])
row = csr.fetchone()
if row:
    print(row)

('Nana', 'Gaines')


In [ ]:
# def read_query(conn, query):
#     csr = conn.cursor()
#     results = None
#     try:
#         csr.execute(query)
#         results = csr.fetchall()
#         # Close and delete cursor
#         csr.close()
#         del csr
#         return results    
#     except Error as err:
#         print(f"Error: '{err}'")

# query = "SELECT TOP 5 first_name, last_name, city FROM sale.customer"
# execute_query(conn, query)
# results = read_query(conn, query)
# print(results)
# conn.close()

# import pandas as pd
# from_db = []

# for result in results:
#     result = list(result)
#     from_db.append(result)

# columns = ['first_name', 'last_name']
# df = pd.DataFrame(from_db, columns=columns)
# display(df)

In [50]:
maxid = csr.execute("select max(brand_id) from product.brand").fetchval()
maxid

40

The fetchone() call returns None. Python then attempts to apply [0] to the result (None[0]) which is not valid.

The fetchval method was created just for this situation - it will detect the fact that there are no rows and will return None.